<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [9]:
import numpy as np
import pandas as pd
import tweepy

import re
import regex
import emoji

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from wordcloud import ImageColorGenerator

from collections import Counter

import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import PIL.Image
from IPython.display import display

import matplotlib.pyplot as plt
%matplotlib inline

init_notebook_mode(connected=True)

In [30]:
keys = pd.read_csv("keys_demo.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [79]:
# Recolectar los datos

tweets = tweepy.Cursor(api.search,
                       q=['Fiona -RT'],
                       lang='es',
                       gc = f"4.649178,-74.062827,70km",
                       tweet_mode='extended',
                       include_rts=False).items(100)


In [80]:
t = []
for tweet in tweets:
        temp = []
        temp.append(tweet.full_text)
        temp.append(tweet.created_at)
        temp.append("Bogotá")
        temp.append(tweet.user.location)
        temp.append(tweet.id)
        temp.append(f"https://twitter.com/i/web/status/{tweet.id}")

        t.append(temp)

In [81]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,"@marttvna @tinacoxina me arrepentí wn, la tina...",2020-11-21 04:44:12,Bogotá,"Santiago, Chile",1330009091582922755,https://twitter.com/i/web/status/1330009091582...
1,"Fiona ladrandole a los fantasmas del hotel, y ...",2020-11-21 04:38:07,Bogotá,Storybrooke,1330007562771705856,https://twitter.com/i/web/status/1330007562771...
2,"Quiero hacer 1 edit de Fiona, pero me da mucha...",2020-11-21 04:33:46,Bogotá,(She/They)|19|🇨🇱,1330006467274022912,https://twitter.com/i/web/status/1330006467274...
3,la temporada diez sin fiona y con la trama de ...,2020-11-21 04:32:29,Bogotá,halliwell manor,1330006143062642689,https://twitter.com/i/web/status/1330006143062...
4,A veces me dan ganas de crearle una cuenta a F...,2020-11-21 04:29:20,Bogotá,,1330005351257825280,https://twitter.com/i/web/status/1330005351257...


In [82]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,"@marttvna @tinacoxina me arrepentí wn, la tina...",2020-11-21 04:44:12,Bogotá
1,"Fiona ladrandole a los fantasmas del hotel, y ...",2020-11-21 04:38:07,Bogotá
2,"Quiero hacer 1 edit de Fiona, pero me da mucha...",2020-11-21 04:33:46,Bogotá
3,la temporada diez sin fiona y con la trama de ...,2020-11-21 04:32:29,Bogotá
4,A veces me dan ganas de crearle una cuenta a F...,2020-11-21 04:29:20,Bogotá


In [83]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [84]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,"@marttvna @tinacoxina me arrepentí wn, la tina...",2020-11-21 04:44:12,Bogotá,"me arrepentí wn, la tina es como fiona, de d...",[],[],"[@marttvna, @tinacoxina]"
1,"Fiona ladrandole a los fantasmas del hotel, y ...",2020-11-21 04:38:07,Bogotá,"Fiona ladrandole a los fantasmas del hotel, y ...",[],[],[]
2,"Quiero hacer 1 edit de Fiona, pero me da mucha...",2020-11-21 04:33:46,Bogotá,"Quiero hacer 1 edit de Fiona, pero me da mucha...",[],[],[]
3,la temporada diez sin fiona y con la trama de ...,2020-11-21 04:32:29,Bogotá,la temporada diez sin fiona y con la trama de ...,[],[],[]
4,A veces me dan ganas de crearle una cuenta a F...,2020-11-21 04:29:20,Bogotá,A veces me dan ganas de crearle una cuenta a F...,[],[],[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [85]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado, ngram_range=(1,1), max_features=100)
tfidf = tfidf_vect.fit_transform(data.clean.values)

In [86]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.T.values)
temp = pd.DataFrame(temp)
temp.columns = voc
temp.index = voc
temp

,abrazar,acaba,agudos,ahora,alguien,aludiendo,amigos,apple,argentinos,ay,...,tomé,ustedes,va,ve,ver,vez,vida,vidas,voy,álbum
abrazar,1.0,0.000000,1.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
acaba,0.0,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.110381,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.189497
agudos,1.0,0.000000,1.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
ahora,0.0,0.000000,0.0,1.000000,0.0,0.0,0.175624,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.066484,0.000000
alguien,0.0,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.183059,0.0,0.145698,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vez,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.151952,0.0,0.326858,1.0,0.000000,0.000000,0.000000,0.000000
vida,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.507602,0.034443,0.0,0.000000,0.0,1.000000,0.080487,0.000000,0.000000
vidas,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.178857,0.0,0.0,...,0.0,0.000000,0.076780,0.0,0.000000,0.0,0.080487,1.000000,0.000000,0.307054
voy,0.0,0.000000,0.0,0.066484,0.0,0.0,0.191450,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000000


In [88]:
trace = go.Heatmap(z=temp.values.tolist(),
                   x=temp.index.values,
                   y=temp.columns.values,
                   colorscale='Reds')

layout = go.Layout(title='🔥 Mapa de calor entre palabras 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)